# Hyper Parameter Tuning
- hyper parameter : 모델 설정과 관련해 직접 지정할 수 있는 매개변수
- model parameter : 회귀계수(가중치), 절편 등 모델의 학습 대상이 되는 변수

### GridSearchCV

In [20]:
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# 데이터 로드
iris_input, iris_target = load_iris(return_X_y=True)

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 값
params = {
    'n_neighbors': range(1, 13, 2)
}

# 첫 번째 인자: 모델
# 두 번째 인자: 테스트 할 파라미터 (딕셔너리)
# scoring: 평가 지표 (accuracy, precision, recall, f1)
# cv: 반복 횟수
grid = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid.fit(iris_input, iris_target)

print("최적의 파라미터:", grid.best_params_)
print("최적화된 모델 객체:", grid.best_estimator_)
print("최적화된 점수:", grid.best_score_)

최적의 파라미터: {'n_neighbors': 7}
최적화된 모델 객체: KNeighborsClassifier(n_neighbors=7)
최적화된 점수: 0.9800000000000001


In [21]:
best_knn = grid.best_estimator_
best_knn.fit(iris_input, iris_target)
best_knn.score(iris_input, iris_target)

0.9733333333333334

### RandomSearchCV
- 하이퍼 파라미터의 값 목록이나 값의 범위를 제공하는데, 이 범위 중에 랜덤하게 값을 뽑아내 최적의 하이퍼 파라미터 조합을 찾는다.
    - 탐색범위가 넓을 때 짧은 시간 내에 좋은 결과를 얻을 수 있다.
    - 랜덤하게 값을 추출해 계산하므로, 전역 최적값을 놓칠 수 있다.

In [22]:
from sklearn.model_selection import RandomizedSearchCV

# 모델 생성
knn = KNeighborsClassifier()

# 테스트할 파라미터 생성
params = {
    'n_neighbors': range(1, 100, 2)
}

# n_iter: 탐색할 최적의 하이퍼 파라미터 조합 수 (기본값: 10)
#         값이 크면 시간이 오래 걸림 / 값이 작으면 좋은 조합을 찾을 가능성 저하
rd_search = RandomizedSearchCV(knn, params, cv=5, n_iter=10, random_state=0)
rd_search.fit(iris_input, iris_target)

print("최적의 파라미터:", rd_search.best_params_)
print("최적화된 모델 객체:", rd_search.best_estimator_)
print("최적화된 점수:", rd_search.best_score_)
rd_search.cv_results_

최적의 파라미터: {'n_neighbors': 5}
최적화된 모델 객체: KNeighborsClassifier()
최적화된 점수: 0.9733333333333334


{'mean_fit_time': array([0.00040026, 0.00020518, 0.00040569, 0.00066199, 0.00060868,
        0.00021157, 0.00068545, 0.00066333, 0.00061707, 0.00029612]),
 'std_fit_time': array([0.00049021, 0.00041037, 0.00049687, 0.00090567, 0.00049717,
        0.00042315, 0.00056988, 0.00054605, 0.00050395, 0.00059223]),
 'mean_score_time': array([0.00241051, 0.00203671, 0.00206203, 0.00498385, 0.00147281,
        0.00201802, 0.00590882, 0.00569768, 0.00153999, 0.00171299]),
 'std_score_time': array([0.00099821, 0.00031915, 0.00108434, 0.00108734, 0.0004674 ,
        0.00063302, 0.00109731, 0.00128452, 0.0004451 , 0.00057985]),
 'param_n_neighbors': masked_array(data=[57, 23, 21, 83, 5, 55, 77, 63, 45, 9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value=999999),
 'params': [{'n_neighbors': 57},
  {'n_neighbors': 23},
  {'n_neighbors': 21},
  {'n_neighbors': 83},
  {'n_neighbors': 5},
  {'n_neighbors': 55},
  {'n_neighb

---

### HyperOpt

**hyper.hp클래스**
<table border="1">
  <thead>
    <tr>
      <th>함수명</th>
      <th>설명</th>
      <th>사용 방법</th>
      <th>예시 코드</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>hp.uniform</td>
      <td>연속적인 실수 값 샘플링</td>
      <td>hp.uniform(label, low, high)</td>
      <td><code>hp.uniform('learning_rate', 0.01, 0.1)</code></td>
    </tr>
    <tr>
      <td>hp.quniform</td>
      <td>연속적이지만 일정 간격(q)을 갖는 값 샘플링</td>
      <td>hp.quniform(label, low, high, q)</td>
      <td><code>hp.quniform('num_layers', 1, 5, 1)</code></td>
    </tr>
    <tr>
      <td>hp.loguniform</td>
      <td>로그 스케일로 분포된 실수 값 샘플링</td>
      <td>hp.loguniform(label, low, high)</td>
      <td><code>hp.loguniform('reg_param', -3, 0)</code></td>
    </tr>
    <tr>
      <td>hp.randint</td>
      <td>정수 값 샘플링</td>
      <td>hp.randint(label, upper)</td>
      <td><code>hp.randint('num_trees', 1, 100)</code></td>
    </tr>
    <tr>
      <td>hp.choice</td>
      <td>주어진 리스트 중 임의의 값 샘플링</td>
      <td>hp.choice(label, options)</td>
      <td><code>hp.choice('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
    </tr>
    <tr>
      <td>hp.normal</td>
      <td>정규분포에서 값 샘플링</td>
      <td>hp.normal(label, mean, std)</td>
      <td><code>hp.normal('dropout_rate', 0.3, 0.05)</code></td>
    </tr>
    <tr>
      <td>hp.lognormal</td>
      <td>로그 정규분포에서 값 샘플링</td>
      <td>hp.lognormal(label, mean, std)</td>
      <td><code>hp.lognormal('scale', 0, 1)</code></td>
    </tr>
  </tbody>
</table>

In [23]:
# !pip install hyperopt

In [24]:
from hyperopt import hp

# 검색 공간
search_space = {
    'x': hp.quniform('x', -10, 10, 1),
    'y': hp.quniform('y', -15, 15, 1)
}

In [25]:
import hyperopt

# 목적 함수
def objective(search_space):
    x = search_space['x']
    y = search_space['y']
    return {
        'loss': x**2 + 20 * y,
        'status': hyperopt.STATUS_OK
    }

In [26]:
from hyperopt import fmin, tpe, Trials

# 탐색 과정을 저장하는 객체
trials = Trials()

# fmin() : 목적 함수의 최소값을 찾는 함수
best_val = fmin(
    fn=objective,       # 목적함수
    space=search_space, # 검색공간
    algo=tpe.suggest,   # 베이지안 최적화 적용
    max_evals=500,      # 반복 횟수
    trials=trials       # 탐색과정 저장
)
best_val

100%|██████████| 500/500 [00:06<00:00, 82.12trial/s, best loss: -300.0] 


{'x': -0.0, 'y': -15.0}

In [27]:
# 탐색과정 -> 목적함수 반환값 (loss와 실행 상태) 저장
trials.results

# 탐색과정 -> 하이퍼 파라미터값을 딕셔너리(리스트) 형태로 저장
trials.vals

{'x': [-0.0,
  -2.0,
  3.0,
  -5.0,
  -1.0,
  -8.0,
  -8.0,
  5.0,
  -4.0,
  6.0,
  -5.0,
  6.0,
  6.0,
  6.0,
  2.0,
  -2.0,
  1.0,
  -3.0,
  -1.0,
  -9.0,
  4.0,
  10.0,
  9.0,
  8.0,
  4.0,
  4.0,
  8.0,
  3.0,
  1.0,
  4.0,
  1.0,
  -0.0,
  2.0,
  1.0,
  -2.0,
  8.0,
  -6.0,
  -1.0,
  3.0,
  -6.0,
  5.0,
  7.0,
  -3.0,
  2.0,
  0.0,
  10.0,
  5.0,
  -4.0,
  7.0,
  3.0,
  -7.0,
  -1.0,
  -3.0,
  5.0,
  1.0,
  -2.0,
  7.0,
  9.0,
  2.0,
  -4.0,
  4.0,
  0.0,
  6.0,
  -5.0,
  9.0,
  5.0,
  4.0,
  2.0,
  -1.0,
  -1.0,
  -1.0,
  -2.0,
  -10.0,
  -3.0,
  1.0,
  -6.0,
  -8.0,
  -5.0,
  -0.0,
  -4.0,
  -2.0,
  -1.0,
  1.0,
  3.0,
  -3.0,
  0.0,
  -1.0,
  1.0,
  -5.0,
  3.0,
  -7.0,
  -3.0,
  2.0,
  1.0,
  -2.0,
  -4.0,
  -1.0,
  -7.0,
  -2.0,
  2.0,
  -4.0,
  0.0,
  3.0,
  -1.0,
  5.0,
  4.0,
  -9.0,
  -5.0,
  -6.0,
  6.0,
  2.0,
  1.0,
  4.0,
  -0.0,
  -2.0,
  1.0,
  -3.0,
  3.0,
  5.0,
  -1.0,
  -4.0,
  2.0,
  -8.0,
  1.0,
  -2.0,
  -3.0,
  0.0,
  2.0,
  7.0,
  -6.0,
  4.0,
  -1.0,
  8.0

- hyperopt를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [ ]:
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from hyperopt import fmin, tpe, Trials, hp
import hyperopt

data = load_breast_cancer()
X_train, X_test, y_train, y_tset = \
    train_test_split(data.data, data.target, random_state=42)

# 1. 검색 공간
search_space = {
    'n_estimators': hp.quniform('n_estimators', 100, 500, 100),
    'max_depth': hp.quniform('max_depth', 3, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
}

# 2. 목적 함수
def xgb_objective(ss):
    xgb_clf = XGBClassifier(
        n_estimators=int(ss['n_estimators']),
        max_depth=int(ss['max_depth']),
        learning_rate=ss['learning_rate'],
        colsample_bytree=ss['colsample_bytree']
    )
    mean_acc = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()
    return {
        'loss': -1 * mean_acc,
        'status': hyperopt.STATUS_OK
    }

# 3. Trials() + fmin()
trials = Trials()
best = fmin(
    fn=xgb_objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

best

100%|██████████| 50/50 [00:13<00:00,  3.77trial/s, best loss: -0.9694835680751174]


{'colsample_bytree': 0.5984060653941006,
 'learning_rate': 0.10551732695927349,
 'max_depth': 10.0,
 'n_estimators': 400.0}

---

### Optuna

<table border="1">
    <thead>
        <tr>
            <th>함수명</th>
            <th>설명</th>
            <th>사용 방법</th>
            <th>예시 코드</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>suggest_uniform</td>
            <td>연속적인 실수 값 샘플링</td>
            <td>trial.suggest_uniform(name, low, high)</td>
            <td><code>trial.suggest_uniform('learning_rate', 0.01, 0.1)</code></td>
        </tr>
        <tr>
            <td>suggest_discrete_uniform</td>
            <td>연속적이지만 일정 간격(step)을 갖는 값 샘플링</td>
            <td>trial.suggest_discrete_uniform(name, low, high, step)</td>
            <td><code>trial.suggest_discrete_uniform('num_layers', 1, 5, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_loguniform</td>
            <td>로그 스케일로 분포된 실수 값 샘플링</td>
            <td>trial.suggest_loguniform(name, low, high)</td>
            <td><code>trial.suggest_loguniform('reg_param', 1e-3, 1)</code></td>
        </tr>
        <tr>
            <td>suggest_int</td>
            <td>정수 값 샘플링</td>
            <td>trial.suggest_int(name, low, high, step)</td>
            <td><code>trial.suggest_int('num_trees', 1, 100)</code></td>
        </tr>
        <tr>
            <td>suggest_categorical</td>
            <td>주어진 리스트 중 임의의 값 샘플링</td>
            <td>trial.suggest_categorical(name, choices)</td>
            <td><code>trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])</code></td>
        </tr>
        <tr>
            <td>suggest_float</td>
            <td>연속적인 실수 값 샘플링 (<code>step</code> 사용 가능)</td>
            <td>trial.suggest_float(name, low, high, step=None, log=False)</td>
            <td><code>trial.suggest_float('alpha', 0.1, 1.0, step=0.1)</code></td>
        </tr>
    </tbody>
</table>

In [30]:
# !pip install optuna

In [31]:
import optuna

# 목적 함수
def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    y = trial.suggest_uniform('y', -15, 15)
    return (x - 3) ** 2 + (y + 5) ** 2

# 스터디 생성
study = optuna.create_study(direction="minimize")

# 최적화 실행
study.optimize(objective, n_trials=500)

# 결과 확인
print(study.best_value)
print(study.best_params)

[I 2025-02-11 17:11:28,477] A new study created in memory with name: no-name-b3387363-9470-4992-a36e-6bbdd279b00f
C:\Users\USER\AppData\Local\Temp\ipykernel_14020\3055804461.py:5: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

C:\Users\USER\AppData\Local\Temp\ipykernel_14020\3055804461.py:6: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2025-02-11 17:11:28,479] Trial 0 finished with value: 32.60568844028214 and parameters: {'x': -2.7101315798006116, 'y': -4.990738161850494}. Best is trial 0 with value: 32.60568844028214.
[I 2025-02-11 17:11:28,480] Trial 1 finished with value: 135.3824968620445 and parameters: {'x': 4.73451615194481, 'y': 6.505387893534362}. Best is trial 0 with value: 32.60568844028214

0.0008819840623159726
{'x': 2.991391542407203, 'y': -4.971576796095637}


In [32]:
import optuna.visualization as vis

# 하이퍼 파라미터 중요도 시각화
vis.plot_param_importances(study).show()

In [33]:
# 최적화 히스토리 시각화
vis.plot_optimization_history(study).show()

- optuna를 활용한 XGBoost 하이퍼 파라미터 튜닝

In [35]:
# 1. 목적 함수
def xgb_optuna_objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
    xgb_clf = XGBClassifier(**params)
    return cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3).mean()

# 2. study 객체 -> 최적화
study = optuna.create_study(direction='maximize')
study.optimize(xgb_optuna_objective, n_trials=50)

# 3. 결과 출력
print(study.best_params)
print(study.best_value)

[I 2025-02-11 17:12:38,775] A new study created in memory with name: no-name-6568cbc0-3404-469a-a9bc-a000674c3dc0
C:\Users\USER\AppData\Local\Temp\ipykernel_14020\2008480700.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-02-11 17:12:38,892] Trial 0 finished with value: 0.9694835680751174 and parameters: {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 0.15362951493172017, 'colsample_bytree': 0.5139458392166949}. Best is trial 0 with value: 0.9694835680751174.
C:\Users\USER\AppData\Local\Temp\ipykernel_14020\2008480700.py:4: FutureWarning:

suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.

[I 2025-02-11 17:12:39,193] Trial 1 finished with value: 0.960093896713615 and parameters: {'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.16251500941572883, 'colsample_bytree': 0.5803348588187184}. Best is trial 0 with value: 0.969483568

{'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.19913145034974566, 'colsample_bytree': 0.5513586454881295}
0.971830985915493


##### HyperOpt vs Optuna

- HyperOpt
    - 'colsample_bytree': 0.5984060653941006
    - 'learning_rate': 0.10551732695927349
    - 'max_depth': 10.0
    - 'n_estimators': 400.0

- Optuna
    - 'n_estimators': 500
    - 'max_depth': 7
    - 'learning_rate': 0.19913145034974566
    - 'colsample_bytree': 0.5513586454881295

In [37]:
from sklearn.metrics import accuracy_score

xgb_hpopt = XGBClassifier(
    n_estimators=400,
    max_depth=10,
    learning_rate=0.11,
    colsample_bytree=0.6
)

xgb_optuna = XGBClassifier(
    n_estimators=500,
    max_depth=7,
    learning_rate=0.2,
    colsample_bytree=0.55
)

xgb_hpopt.fit(X_train, y_train)
xgb_optuna.fit(X_train, y_train)

hpopt_pred = xgb_hpopt.predict(X_test)
optuna_pred = xgb_optuna.predict(X_test)

print(f"HyperOpt 최적 파라미터 적용: {accuracy_score(y_tset, hpopt_pred)}")
print(f"Optuna 최적 파라미터 적용: {accuracy_score(y_tset, optuna_pred)}")

HyperOpt 최적 파라미터 적용: 0.958041958041958
Optuna 최적 파라미터 적용: 0.965034965034965
